# Работа О.3. Дифракция на шероховатой поверхности
#### (по курсу "Цифровизация физических процессов")


### 0. Подготовка к работе
#### Импорт библиотек

In [ ]:
import numpy as np
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import display, Math, Latex
import os
plt.rcParams['figure.dpi'] = 100 # размер изображений и графиков

# словари для сохранения результатов расчётов
results_ind = {}
results_cov = {}

#### Рабочая папка
Задайте имя рабочей папки в переменной FOLDER. Папка должна располагаться внутри папки "Рабочий стол\спеклы\".

In [ ]:
# Задайте имя рабочей папки (используйте шаблон Группа_Фамилия) 
# Например: 'Б01-001_Иванов'
FOLDER = '1-001_Иванов'

# формат изображений
IMGFORMAT = 'png'

WD = os.path.join(os.environ['USERPROFILE'], 'Desktop', 'спеклы', FOLDER)
if os.path.exists(WD):
    print('Обнаружена рабочая папка:\n', WD)
    print(f'Список файлов (изображения *.{IMGFORMAT}):')
    FILELIST = [os.path.basename(f) for f in glob(os.path.join(WD, '*.' + IMGFORMAT))]
    FILELIST.sort()
    print('\n'.join([f'{i}: {f}' for i, f in enumerate(FILELIST)]))
else:
    print(f'Ошибка! Папка {WD} не существует! Поместите фотографии спеклов\n'
          f'в папку "Рабочий стол\\спеклы\\[Группа_Фамилия]" и перезапустите блок')
    raise RuntimeError

#### Функции обработки изображений

In [ ]:
def get_image(filename):
    ''' Получить изображение из файла и преобразовать его красный канал в 2D-массив'''
    with Image.open(os.path.join(WD, filename)) as img:
        print('\n\n')
        print("Исходное изображение: ", filename)
        plt.imshow(img)
        plt.show()
    return np.array(img)[:,:,0] # используем только *красный* канал


def get_center(data):
    ''' Определить "центр масс" изображения'''
    Y = np.arange(data.shape[0]) # Y -- строки
    X = np.arange(data.shape[1]) # X -- столбцы
    S = np.sum(data)
    X0 = round(X @ np.sum(data, axis=0) / S)
    Y0 = round(Y @ np.sum(data, axis=1) / S)
    print(f"Центр: {X0 = }, {Y0 = }")
    return X0, Y0


def crop_centered(data, mass=False, less=0):
    ''' Вырезать из изображения квдрат по центру

        mass: если True, вычислить положение центра масс, иначе использовать геометрический центр
        less: количество пикселей, на которое уменьшить размер обрезающего квадрата 
    '''
    NY, NX = data.shape
    if mass:
        # координаты "центра масс"
        X0, Y0 = get_center(data)
    else:
        # взять геометрический центр рамки
        X0 = NX // 2
        Y0 = NY // 2
    W = min(X0, Y0, NX - X0, NY - Y0) - less
    print(f"Ширина = {2 * W}")
    y1 = Y0 - W
    y2 = Y0 + W
    x1 = X0 - W
    x2 = X0 + W
    print(f"Обрезка: [{y1}:{y2}, {x1}:{x2}]")
    cropped = data[y1:y2, x1:x2]
    
    print("Обрезанное изображение:")
    plt.imshow(cropped, cmap='gray')
    plt.show()
    plt.title("Гистограмма")
    plt.hist(data.flatten(), bins=256, histtype='step', log=True)
    plt.show()
    return cropped

## 1. Вычисление радиуса индикатрисы

#### Функция вычисления индикатрисы

In [ ]:
def indicatrice(data, name=''):
    ''' Усреднение интенсивности как функции радиуса;
        Построение графика в координатах log I (r^2) и определение наклона наилучшей прямой.
    '''
    NN = 100 # число разбиений по радиусу
    X0 = data.shape[1] // 2
    Y0 = data.shape[0] // 2
    R2 = np.linspace(0., X0**2, NN + 1, dtype=int)
    I = np.zeros(NN - 1, dtype=np.int64)
    sigmaI = np.zeros(NN - 1, dtype=float)
    print('Подготавливаем данные...')
    
    # квадраты координат (относительно центра)
    dx2 = np.arange(-X0, X0, dtype=np.int64)**2
    dy2 = np.arange(-X0, X0, dtype=np.int64)**2
    
    # квадраты радиусов всех точек (матрица a x a)
    dr2 = dx2[:, np.newaxis] + dy2[np.newaxis, :]
    
    # раскладка радиусов по "корзинам"
    R2n = np.digitize(dr2, bins=R2)
    R2 = R2[:-2] # обрезаем точки, выпадающие за максимальный радиус
    
    # Суммирование статистики:
    # определяем, к какому диапазону радиусов принадлежит каждая точка и добавляем туда её интенсивность 
    print('Суммируем статистику...')
    statI = {}
    for i in range(data.size):         
        n = R2n.flat[i] - 1
        if n < NN - 1: # отбрасываем точки за пределами большого круга            
            if n in statI:
                statI[n].append(int(data.flat[i]))
            else:
                statI[n] = [int(data.flat[i])]

    for n in statI.keys():
        I[n] = np.mean(statI[n]) # средняя интенсивность в ячейке n
        sigmaI[n] = np.std(statI[n]) / np.sqrt(len(statI[n])) # среднеквадратичное отклонение
    
    print('Строим график:')
    logI = np.log(I)
    # погрешности 
    sigma_logI = sigmaI / I
    sigma_R = 1  # +- пиксель
    sigma_R2 = 2 * np.sqrt(R2) * sigma_R  # sigma(x^2) = 2 * x * sigma(x) 
    
    # аппроксимируем график функцией вида log I = k * r^2 + b
    func = lambda x, k, b: k * x + b
    popt, pcov = curve_fit(func, R2, logI, sigma = sigma_logI)    
    # погрешность углового коэффициента
    sigma_k = np.sqrt(pcov[0,0])

    plt.title(f"Зависимость логарифма интенсивности от квадрата радиуса ({name})")
    plt.xlabel("$R^2$")
    plt.ylabel("log I")
    plt.errorbar(R2, logI, xerr=sigma_R2, yerr=sigma_logI, fmt='none')
    #plt.plot(R2, logI, '.')
    plt.plot(R2, func(R2, *popt), 'r-')
    plt.show()
    plt.hist
    
    print("Угловой коэффициент:")
    print(f"k = {popt[0]:.3e} +- {sigma_k:.1e} ({np.abs(sigma_k/popt[0]) * 100:.1f}%)")
    
    return popt[0], sigma_k

#### Расчёт радиусов индикатрисы

Введите список фалов для вычисления индикатрисы (мелкие спеклы) в переменную IND_FILES и запустите блок.

Изучите вид полученных графиков. Если результаты удовлетворительно аппроксимируются теоретической кривой $y=Ae^{kx^2}$ ($\log y = k x^2 + b$), запишите значение углового коэффициента $k$ и его погрешность $\sigma_k$. Результаты сохраняются в переменной results_ind

In [ ]:
# Введите список НОМЕРОВ файлов изображений для вычисления индикатрисы мелких спектров
IND_FILES = [0, ]

IND_FILENAMES = [FILELIST[i] for i in IND_FILES]
print("Файлы: ", IND_FILENAMES)
for f in IND_FILENAMES:
    results_ind[f] = indicatrice(crop_centered(get_image(f), mass=True), name=f)

In [ ]:
print(results_ind)

### Задание для самостоятельной обработки
* По найденным значениям $k$ вычислите радиусы индикатрисы $\rho_i$ в пикселях и его погрешность $\sigma_{\rho_i}$
* Пересчитайте размеры в пикселях в реальные линейные размеры (мм)
* Определите радиус корреляции фазы $\rho_A$ и его погрешность $\sigma_{\rho_A}$
* Определите степень шероховатости пластинки $\zeta$

## 2. Вычисление функции и радиуса корреляции
##### Загрузка изображения
Введите имя файла filename изображения для расчёта радиуса корреляции картины. 

Проведите расчёт отдельно для каждого файла.

In [ ]:
# введите номер файла в списке файлов рабочей папки
filename = FILELIST[0]

# Загрузка и обрезка изображения
M = crop_centered(get_image(filename), mass=False).astype(np.int32)

### 2.1 Расчёт функции корреляции
##### Функция  корреляции

In [ ]:
# Параметры расчёта
MIN_RHO = 10      # минимальное смещение (по умолчанию 10)
MAX_RHO = 100     # максимальное смещение (по умолчанию 100)
MIN_GAMMA = 0.2   # минимальный порог значения функции корреляции (по умолчанию 0.2)
    
def shift_matrix(M, dr):
    ''' Смещение матрицы на вектор dr = (dx, dy) с обрезанием.
        Возвращает пару (M1, M2), где M1 -- смещённая матрица, M2 -- исходная обрезанная матрица. 
    '''
    dx, dy = dr
    if dx < 0 or dy < 0 or dx > M.shape[0] or dy > M.shape[1]:
        raise ValueError
    return M[dx:, dy:], M[:M.shape[0] - dx, :M.shape[1] - dy]
    
    
def correlation(M, direction):
    ''' Расчёт степени корреляции gamma(rho).
        Возвращает два массива точек (rho, gamma)
    '''
    N = (MAX_RHO + 1) // max(direction)
    
    # создаем массив из векторов смещений
    # Пример: direction = [1,2] -> dr = [[0,0], [1,2], [2,4], [3,6], ...]
    dr = np.array(direction) * np.arange(N)[:, np.newaxis]
    
    # Радиусы смещений
    rho = np.sqrt(dr[:,0]**2 + dr[:,1]**2)
    
    gamma = np.zeros(N, dtype=float)
    gamma[0] = 1.0

    meanM = np.mean(M) # среднее
    varM = np.var(M)   # дисперсия
    dM = M - meanM     # отклонение
    i = 0
    while i < N - 1:
        i += 1
        # смещение по X
        dM1, dM2 = shift_matrix(dM, dr[i])
        gamma[i] = np.mean(dM1 * dM2) / np.var(dM1)
        if (gamma[i] < MIN_GAMMA and rho[i] > MIN_RHO):
            break
    return rho[:i], gamma[:i]

#### Вычисление функции корреляции
Функция корреляции вычисляется по 3-м осям: смещение вдоль оси $x$, вдоль оси $y$ и вдоль диагонали $x=y$

In [ ]:
CORRELATIONS = [(correlation(M, (0, 1)), 'v', 'смещение по X'),
                (correlation(M, (1, 0)), '^', 'смещение по Y'),
                (correlation(M, (1, 1)), '>', 'смещение по X = Y'),]
for (X, Y), marker, label in CORRELATIONS:
    plt.plot(X, Y, marker + '--', label=label)
plt.title(f"Зависимость степени корреляции от смещения ({filename})")
plt.xlabel(r"$\rho$, [пикс]")
plt.ylabel(r"$\gamma$")
plt.xlim(0)
plt.legend()
plt.show()

### 2.2 Расчёт радиуса корреляции
Каждая функция корреляции $\gamma(\rho)$ аппроксимируется гауссовой кривой $\gamma=Ae^{k\rho^2}$ ($\log \gamma = k\rho^2 + b$).

Реализована возможность исключить из расчёта несколько начальных (малые $\rho$) и конечных (большие $\rho$) точек.

##### Функция аппроксимации

In [ ]:
def approximate(x, y, label=''):
    cut_start = 0  # обрезка в начале
    cut_end = 0    # обрезка в конце
    while True:
        x_cut = x[cut_start:x.size - cut_end]
        y_cut = y[cut_start:y.size - cut_end]
        
        # аппроксимируем график функцией вида Y = k * X + b
        popt, pcov = curve_fit(lambda X, k, b: k * X + b, x_cut, y_cut)
        # погрешность углового коэффициента (по МНК)
        sigma_k = np.sqrt(pcov[0,0])

        plt.plot(x_cut, y_cut, '+')
        plt.plot(x_cut, popt[0] * x_cut + popt[1])
        plt.xlabel(r'$\rho^2$')
        plt.ylabel(r'$\log \gamma$')
        plt.title(f'Функция корреляции ({filename}:{label}, обрезка [{cut_start}:-{cut_end}])')
        plt.show()

        print(f"k = {popt[0]:.3e} +- {sigma_k:.1e} ({np.abs(sigma_k/popt[0]) * 100:.1f}%)")

        try:
            print("\nУточнение расчёта. Чтобы завершить расчёт, оставьте поле ввода пустым и нажмите Enter")
            print("(чтобы вернуть точки, введите отрицательное число)")
            cut_start += int(input("Сколько точек убрать в начале (слева)? "))
            cut_end += int(input("Сколько точек убрать в конце (справа)? "))
        except ValueError:  # закончить, если введено не число или пустая строка
            break
            
    print("Аппроксимация завершена.\n\n")
    return popt[0], sigma_k

#### Запуск расчёта
Для каждой полученной выше функции корреляции определяется наилучший угловой коэффициент прямой.

Рассмотрите получающиеся графики и качественно сделайте вывод о том, насколько хорошо они могут быть приближены прямой линией. 

Если точки плохо ложаться на прямую, следуя инструкциям, задайте ограничения диапазона точек (обрезку слева и справа), используемых при аппроксимации. 

**Замечание**. Для получения корректных результатов наклон следует определять в пределе _малых_ $\rho$. Точки слева (в начале графика) рекомендуется отбрасывать, только если на изображении спеклов видны мелкомасштабные интерференционные полосы.  

In [ ]:
if filename in results_cov:
    print(f"Внимание: результаты расчёта для {filename} будут перезаписаны")
    if input("Продолжить (Д/н)?").upper() in ['Н', 'N']:
        raise RuntimeError

results_cov[filename] = []
current_results = results_cov[filename]
for (RHO, GAMMA), _, label in CORRELATIONS:
    print("-----------------------------------------------------")
    print(f'Расчёт функции корреляции ln Г(rho^2) для "{label}"')
    current_results.append(approximate(RHO**2, np.log(GAMMA), label))
print("***************")
print("Результаты:")
k = np.array(current_results)[:,0]
sigma_k = np.array(current_results)[:, 1]
for i in range(len(current_results)):
    print(f"{CORRELATIONS[i][2]}:\t k = {k[i]:.3e} +- {sigma_k[i]:.1e}")

mean_k = np.mean(k)
var_k = np.var(k)
sigma_mean_k = np.sqrt(var_k + sigma_k @ sigma_k)
current_results.append((mean_k, sigma_mean_k))
print(f'Среднее: <k> = {mean_k:.3e} +- {sigma_mean_k:.1e}')

In [ ]:
print(results_cov)

### Задание для самостоятельной обработки
* По найденным значениям $k$ вычислите радиусы корреляции $\rho_{с}$ в пикселях и погрешности $\sigma_{\rho_{с}}$
* Пересчитайте размеры в пикселях в реальные линейные размеры (мм)
* Определите диаметр лазерного луча $D$ и погрешность $\sigma_D$
* Проанализируйте зависимость диаметра луча $D$ от расстояния $L_1$ между пластинкой и фокусом линзы